In [4]:
import re
import pandas

In [ ]:
NODE_DATA = {
    "Cray EX255a": {
        "sockets": 4,
        "accelerators": 4,
    },
    "Cray EX235a": {
        "sockets": 2,
        "accelerators": 4,
    },
    "Cray EX254n": {
        "sockets": 4,
        "accelerators": 4,
    },
    "Intel Exascale Compute Blade": {
        "sockets": 2,
        "accelerators": 6,
    },
    "NDv5": {
        "sockets": 2,
        "accelerators": 8,
    },
    re.compile("ND96_[^S+]_v4"): {
        "sockets": 2,
        "accelerators": 8,
    },
    "Fugaku": {
        "sockets": 1,
        "accelerators": 0,
    },
    "AC922": {
        "sockets": 2,
        "accelerators": 6,
    },
    "NVIDIA DGX H100": {
        "sockets": 2,
        "accelerators": 8,
    },
    "BullSequana XH2000": {
        "sockets": 2,
        "accelerators": 4,
    },
    "BullSequana XH3000": {
        "sockets": 4,
        "accelerators": 4,
    },
}

def sockets_per_node(row):
    for node_type, values in NODE_DATA.items():
        if isinstance(node_type, str) and node_type in row["System Model"]:
            return values["sockets"]
        if isinstance(node_type, re.Pattern) and node_type.match(row["System Model"]):
            return values["sockets"]
    return None

def accelerators_per_node(row):
    for node_type, values in NODE_DATA.items():
        if isinstance(node_type, str) and node_type in row["System Model"]:
            return values["accelerators"]
        if isinstance(node_type, re.Pattern) and node_type.match(row["System Model"]):
            return values["accelerators"]
    return None

In [31]:
top500 = pandas.read_excel('TOP500_202411.xlsx')

top500['Non-accelerator Cores'] = top500['Total Cores'] - top500['Accelerator/Co-Processor Cores'].fillna(0)
top500['Sockets/node'] = top500.apply(sockets_per_node, axis=1)
top500['Accelerators/node'] = top500.apply(accelerators_per_node, axis=1)
top500['Nodes'] = top500['Non-accelerator Cores'] / top500['Cores per Socket'] / top500['Sockets/node']
top500['Non-accelerator Sockets'] = top500['Non-accelerator Cores'] / top500['Cores per Socket']
top500['Accelerator Cores'] = (top500['Accelerator/Co-Processor Cores'] / top500['Nodes'] / top500['Accelerators/node']).fillna(0)
top500['Rmax/node'] = top500['Rmax [TFlop/s]'] / top500['Nodes']
top500['Rmax/accelerator'] = (top500['Rmax/node'] / top500['Accelerators/node']).replace([float('inf'), float('-inf')], 0)
top500['Rpeak/node'] = top500['Rpeak [TFlop/s]'] / top500['Nodes']
top500['Rpeak/accelerator'] = top500['Rpeak/node'] / top500['Accelerators/node'].replace([float('inf'), float('-inf')], 0)
top500['HPL Seconds'] = ((2.0 / 3.0 * top500['Nmax'] ** 3 + 2 * top500['Nmax'] ** 2) / 1e12) / top500['Rmax [TFlop/s]']
top500

,Rank,Previous Rank,First Appearance,First Rank,Name,Computer,Site,Manufacturer,Country,Year,...,Sockets/node,Accelerators/node,Nodes,Non-accelerator Sockets,Accelerator Cores,Rmax/node,Rmax/accelerator,Rpeak/node,Rpeak/accelerator,HPL Seconds
0,1,NaN,64,1,El Capitan,"HPE Cray EX255a, AMD 4th Gen EPYC 24C 1.8GHz, ...",DOE/NNSA/LLNL,HPE,United States,2024,...,4.0,4.0,10952.0,43808.0,228.0,159.057706,39.764427,250.764800,62.691200,6305.887875
1,2,1.0,59,1,Frontier,"HPE Cray EX235a, AMD Optimized 3rd Generation ...",DOE/SC/Oak Ridge National Laboratory,HPE,United States,2021,...,2.0,4.0,4802.0,9604.0,440.0,281.757601,70.439400,428.096000,107.024000,7549.439405
2,3,2.0,62,2,Aurora,"HPE Cray EX - Intel Exascale Compute Blade, Xe...",DOE/SC/Argonne National Laboratory,Intel,United States,2023,...,2.0,6.0,10624.0,21248.0,128.0,95.256024,15.876004,186.371047,31.061841,15693.648462
3,4,3.0,62,3,Eagle,"Microsoft NDv5, Xeon Platinum 8480C 48C 2GHz, ...",Microsoft Azure,Microsoft Azure,United States,2023,...,2.0,8.0,1800.0,3600.0,132.0,311.777778,38.972222,470.464000,58.808000,1950.062345
4,5,NaN,64,5,HPC6,"HPE Cray EX235a, AMD Optimized 3rd Generation ...",Eni S.p.A.,HPE,Italy,2024,...,2.0,4.0,1665.0,3330.0,440.0,287.027027,71.756757,364.544000,91.136000,3758.462316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,452.0,53,104,F1,"Lenovo HR650x, Xeon Gold 6133 20C 2.5GHz, 25G ...",Service Provider T,Lenovo,China,2019,...,NaN,NaN,NaN,2800.0,0.0,NaN,NaN,NaN,NaN,40086.017584
496,497,453.0,56,179,NaN,"Inspur TS10000 HPC Server, Xeon Gold 6226R 16C...",Energy Company B,Inspur,China,2020,...,NaN,NaN,NaN,520.0,0.0,NaN,NaN,NaN,NaN,687.963476
497,498,454.0,55,151,NaN,"ThinkSystem SR590, Xeon Gold 5218 16C 2.3GHz, ...",Service Provider T,Lenovo,China,2020,...,NaN,NaN,NaN,6800.0,0.0,NaN,NaN,NaN,NaN,151949.382145
498,499,455.0,59,301,VSC-5,"MEGWARE SLIDESX, AMD EPYC 7713 64C 2GHz, Infin...",Vienna Scientific Cluster,MEGWARE,Austria,2022,...,NaN,NaN,NaN,1488.0,0.0,NaN,NaN,NaN,NaN,77408.077591
